In [1]:
import keras.optimizers
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#导入必要的库

TypeError: Descriptors cannot be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

In [2]:
#读取降雨径流数据
df = pd.read_csv("/Users/datou/PycharmProjects/model/JianyangSta/JY_data/建阳站日降雨径流蒸发资料(完整）.csv", parse_dates=["Date"])
df_copy = df
df.reset_index(drop=True)
df = df.set_index('Date')
df

,Q,P
Date,,
2000-03-04,95.218583,2.779
2000-03-05,111.431333,2.721
2000-03-06,92.994458,1.991
2000-03-07,90.252708,5.497
2000-03-08,86.227667,4.970
...,...,...
2018-10-26,39.116250,8.492
2018-10-27,46.711250,5.857
2018-10-28,70.970417,3.264


In [3]:
n_past=7

In [4]:
#划分数据
test_split = round(len(df) * 0.30)
df_for_training = df[:-test_split]
df_for_testing = df[-(test_split+n_past):]
df_for_testing

,Q,P
Date,,
2013-03-20,225.427083,23.976
2013-03-21,264.641667,0.224
2013-03-22,179.575000,0.000
2013-03-23,254.883333,8.498
2013-03-24,518.239583,30.304
...,...,...
2018-10-26,39.116250,8.492
2018-10-27,46.711250,5.857
2018-10-28,70.970417,3.264


In [5]:
# 数据归一化
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 1))
df_for_training_scaled = scaler.fit_transform(df_for_training)
df_for_testing_scaled = scaler.transform(df_for_testing)
df_for_training_scaled = pd.DataFrame(df_for_training_scaled, columns=df_for_training.columns)
df_for_testing_scaled = pd.DataFrame(df_for_testing_scaled, columns=df_for_training.columns)
df_for_training_scaled

,Q,P
0,0.013690,0.012989
1,0.016311,0.012718
2,0.013330,0.009306
3,0.012887,0.025693
4,0.012236,0.023230
...,...,...
4766,0.027327,0.000000
4767,0.039502,0.039719
4768,0.082076,0.141640
4769,0.053736,0.000000


In [6]:
#构造训练集测试集 
# 创建数据集，n_past：时间步长
def createXY(dataset, n_past):
    dataX = []
    dataY = []
    for i in range(n_past, len(dataset)):
        dataX.append(np.array(dataset)[i - n_past:i, 0:np.array(dataset).shape[1]])
        dataY.append(np.array(dataset)[i, 0])
    return np.array(dataX), np.array(dataY)

In [7]:
# 定义时间窗


In [8]:
trainX, trainY = createXY(df_for_training_scaled, n_past)
trainX.shape, trainY.shape
len(trainX)

4764

In [9]:
testX, testY = createXY(df_for_testing_scaled, n_past)
testX.shape, testY.shape

((2044, 7, 2), (2044,))

In [10]:
# BP神经网络输入是二维数据！！！一定要将处理过后的三维数据转为二维数据！！！
trainX = trainX.reshape(len(trainX), n_past * 2)

In [11]:
testX = testX.reshape(len(testX), n_past * 2)

In [12]:
#建立bp模型 训练 
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.optimizers import Adam


def build_model():
    model = Sequential()  #层次模型
    model.add(Dense(16, input_dim=n_past*2, kernel_initializer='uniform', activation='relu'))  #输入层，Dense表示BP层
    #model.add(Activation('relu'))  #添加激活函数
    model.add(Dense(64, bias_initializer='uniform', activation='sigmoid'))  #中间层
    #model.add(Activation('sigmoid'))  #添加激活函数
    model.add(Dense(1))  #输出层
    learning_rate = 3e-3
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(loss='mse', optimizer=optimizer)  #编译模型
    return model

# model=build_model()
# 
# model.fit(trainX, trainY, epochs =50, batch_size = 16) #训练模型nb_epoch=50次

In [13]:
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV

# 交叉搜索，寻求模型最佳参数，并输出最佳模型
parameters = {'batch_size': [16, 20],
              'epochs': [16, 20]}
"""
verbose参数设置为 2，可以将显示的信息量减小到每轮训练的损失。
可以通过将verbose设置为 1 来关闭所有输出
使用KerasRegressor来包装Keras模型进行网格搜索和交叉验证
"""
grid_model = KerasRegressor(build_fn=build_model, verbose=1)
grid_search = GridSearchCV(estimator=grid_model, param_grid=parameters,
                           cv=5)

/var/folders/p_/spnxgg1953n3fzj8z81sv51r0000gn/T/ipykernel_42826/2192670547.py:12: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  grid_model = KerasRegressor(build_fn=build_model, verbose=1)


In [14]:
"""
训练网络
"""
grid_search = grid_search.fit(trainX, trainY)
# 将最佳模型保存在my_model变量中
model = grid_search.best_estimator_.model

Epoch 1/16
  1/239 [..............................] - ETA: 27s - loss: 0.0380

2024-03-28 22:11:54.244035: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


239/239 [==============================] - 0s 329us/step - loss: 0.0019
Epoch 2/16
239/239 [==============================] - 0s 305us/step - loss: 8.2345e-04
Epoch 3/16
239/239 [==============================] - 0s 310us/step - loss: 6.8852e-04
Epoch 4/16
239/239 [==============================] - 0s 328us/step - loss: 6.6858e-04
Epoch 5/16
239/239 [==============================] - 0s 319us/step - loss: 7.1287e-04
Epoch 6/16
239/239 [==============================] - 0s 310us/step - loss: 5.9540e-04
Epoch 7/16
239/239 [==============================] - 0s 293us/step - loss: 7.4299e-04
Epoch 8/16
239/239 [==============================] - 0s 290us/step - loss: 6.1513e-04
Epoch 9/16
239/239 [==============================] - 0s 293us/step - loss: 5.9844e-04
Epoch 10/16
239/239 [==============================] - 0s 313us/step - loss: 5.8646e-04
Epoch 11/16
239/239 [==============================] - 0s 314us/step - loss: 5.9240e-04
Epoch 12/16
239/239 [==============================] - 0

In [15]:
grid_search.best_params_

{'batch_size': 20, 'epochs': 16}

In [16]:
model.summary()

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_60 (Dense)            (None, 16)                240       
                                                                 
 dense_61 (Dense)            (None, 64)                1088      
                                                                 
 dense_62 (Dense)            (None, 1)                 65        
                                                                 
Total params: 1,393
Trainable params: 1,393
Non-trainable params: 0
_________________________________________________________________


In [17]:
"""
在测试集上的预测
"""
y_predict = model.predict(testX)

64/64 [==============================] - 0s 219us/step


In [18]:
train_predict = model.predict(trainX)

149/149 [==============================] - 0s 186us/step


In [19]:
# 逆缩放
prediction_copies_array = np.repeat(y_predict, 2, axis=-1)
pred = scaler.inverse_transform(np.reshape(prediction_copies_array, (len(y_predict), 2)))[:, 0]
pred

array([370.65866 , 166.92632 , 148.39894 , ..., -27.485518, -48.927273,
       -83.82938 ], dtype=float32)

In [20]:
pred = pd.DataFrame(pred)
pred.columns = ['Qsim']
pred

,Qsim
0,370.658661
1,166.926315
2,148.398941
3,235.396103
4,160.929993
...,...
2039,-84.213837
2040,-19.184826
2041,-27.485518
2042,-48.927273


In [21]:
# 同样步骤进行逆变换
original_copies_array = np.repeat(testY, 2, axis=-1)
original = scaler.inverse_transform(np.reshape(original_copies_array, (len(testY), 2)))[:, 0]
original = pd.DataFrame(original)
original.columns = ['Qobs']
original

,Qobs
0,357.285417
1,266.375000
2,292.975000
3,279.295833
4,263.750000
...,...
2039,39.116250
2040,46.711250
2041,70.970417
2042,64.495417


In [22]:
# 对测试集输出数据进行逆变换
prediction_copies_array = np.repeat(train_predict, 2, axis=-1)
train_pred = scaler.inverse_transform(np.reshape(prediction_copies_array, (len(train_predict), 2)))[:, 0]
train_pred.shape

(4764,)

In [23]:
# 对测试数据Y进行逆变换
train_original_copies_array = np.repeat(trainY, 2, axis=-1)
train_original = scaler.inverse_transform(np.reshape(train_original_copies_array, (len(trainY), 2)))[:, 0]
train_original.shape

(4764,)

In [24]:
train_pred = pd.DataFrame(train_pred)
train_pred.columns = ['Qsim']
train_pred

,Qsim
0,31.066195
1,146.315338
2,38.648571
3,-1.733313
4,12.048175
...,...
4759,54.205063
4760,-10.140836
4761,53.645103
4762,494.237854


In [25]:
train_original = pd.DataFrame(train_original)
train_original.columns = ['Qobs']
train_original

,Qobs
0,182.373833
1,259.538958
2,168.067458
3,136.108833
4,274.106833
...,...
4759,179.575000
4760,254.883333
4761,518.239583
4762,342.933333


In [26]:
df_copy

,Date,Q,P
0,2000-03-04,95.218583,2.779
1,2000-03-05,111.431333,2.721
2,2000-03-06,92.994458,1.991
3,2000-03-07,90.252708,5.497
4,2000-03-08,86.227667,4.970
...,...,...,...
6810,2018-10-26,39.116250,8.492
6811,2018-10-27,46.711250,5.857
6812,2018-10-28,70.970417,3.264
6813,2018-10-29,64.495417,0.000


In [27]:
test_df = df_copy[-test_split :]
test_df

,Date,Q,P
4771,2013-03-27,357.285417,0.000
4772,2013-03-28,266.375000,5.566
4773,2013-03-29,292.975000,15.177
4774,2013-03-30,279.295833,7.662
4775,2013-03-31,263.750000,0.622
...,...,...,...
6810,2018-10-26,39.116250,8.492
6811,2018-10-27,46.711250,5.857
6812,2018-10-28,70.970417,3.264
6813,2018-10-29,64.495417,0.000


In [28]:
train_df = df_copy[n_past:-test_split]
train_df

,Date,Q,P
7,2000-03-11,182.373833,17.821
8,2000-03-12,259.538958,1.398
9,2000-03-13,168.067458,1.793
10,2000-03-14,136.108833,4.513
11,2000-03-15,274.106833,37.667
...,...,...,...
4766,2013-03-22,179.575000,0.000
4767,2013-03-23,254.883333,8.498
4768,2013-03-24,518.239583,30.304
4769,2013-03-25,342.933333,0.000


In [29]:
test_df = test_df.reset_index(drop=True)
test_df = test_df['Date']
test_df

0      2013-03-27
1      2013-03-28
2      2013-03-29
3      2013-03-30
4      2013-03-31
          ...    
2039   2018-10-26
2040   2018-10-27
2041   2018-10-28
2042   2018-10-29
2043   2018-10-30
Name: Date, Length: 2044, dtype: datetime64[ns]

In [30]:
train_df = train_df.reset_index(drop=True)
train_df = train_df['Date']
train_df

0      2000-03-11
1      2000-03-12
2      2000-03-13
3      2000-03-14
4      2000-03-15
          ...    
4759   2013-03-22
4760   2013-03-23
4761   2013-03-24
4762   2013-03-25
4763   2013-03-26
Name: Date, Length: 4764, dtype: datetime64[ns]

In [31]:
test_df_full = pd.concat([test_df, original, pred], axis=1)
test_df_full = test_df_full.reset_index(drop=True)
test_df_full = test_df_full.set_index('Date')
test_df_full.to_csv('/Users/datou/PycharmProjects/model/JianyangSta/JY_data/JY_TestBP.csv')
test_df_full

,Qobs,Qsim
Date,,
2013-03-27,357.285417,370.658661
2013-03-28,266.375000,166.926315
2013-03-29,292.975000,148.398941
2013-03-30,279.295833,235.396103
2013-03-31,263.750000,160.929993
...,...,...
2018-10-26,39.116250,-84.213837
2018-10-27,46.711250,-19.184826
2018-10-28,70.970417,-27.485518


In [32]:
train_df_full = pd.concat([train_df, train_original, train_pred], axis=1)
train_df_full = train_df_full.reset_index(drop=True)
train_df_full = train_df_full.set_index('Date')
train_df_full.to_csv('/Users/datou/PycharmProjects/model/JianyangSta/JY_data/JY_TrainBP.csv')
train_df_full

,Qobs,Qsim
Date,,
2000-03-11,182.373833,31.066195
2000-03-12,259.538958,146.315338
2000-03-13,168.067458,38.648571
2000-03-14,136.108833,-1.733313
2000-03-15,274.106833,12.048175
...,...,...
2013-03-22,179.575000,54.205063
2013-03-23,254.883333,-10.140836
2013-03-24,518.239583,53.645103


In [33]:
# 计算nash效率系数
from tool import Indicator

mae,rmse,r2,nse,re,qmaxe,qmine = Indicator.eval(test_df_full['Qsim'], test_df_full['Qobs'])

MAE:154.0730115341061
RMSE:198.06950811199238
NSE:0.5117500781522951
RE:-60.95846685266967
Qmaxe:0.09632073208343267
Qmine:6.177052655305271


ValueError: not enough values to unpack (expected 7, got 6)

In [ ]:
# 比较预测值和原始值，进行绘图展示
plt.figure(figsize=(12, 5))
plt.plot(test_df_full['Qobs'], color='red', label='实测值')
plt.plot(test_df_full['Qsim'], color='blue', label='模拟值', linestyle="--")
plt.title(' BP')
plt.xlabel('年份')
plt.ylabel('流量(m3/s)')
plt.legend()
plt.savefig("/Users/datou/PycharmProjects/model/JianyangSta/JY_picture/TestBP.png")
plt.show()

In [ ]:
# 计算nash效率系数
train_mae,train_rmse,train_r2,train_nse,train_re,train_qmaxe,train_qmine = Indicator.eval(train_df_full['Qsim'], train_df_full['Qobs'])
# 比较预测值和原始值，进行绘图展示
plt.figure(figsize=(12, 5))
plt.plot(train_df_full['Qobs'], color='red', label='实测值')
plt.plot(train_df_full['Qsim'], color='blue', label='模拟值', linestyle="--")
plt.title(' BP')
plt.xlabel('年份')
plt.ylabel('流量(m3/s)')
plt.legend()
plt.savefig("/Users/datou/PycharmProjects/model/JianyangSta/JY_picture/TrainBP.png")
plt.show()